<a href="https://colab.research.google.com/github/park1NG/MLTeamProject/blob/main/Aero%20Delay_20230603.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb

from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [22]:
def data_preprocessing(df):
    df.dropna()
    
    df = df.drop(df[df['예상'] == ':'].index)
    df = df.drop(df[df['도착'] == ':'].index)
    df = df.drop(df[df['현황'] == '취소'].index)

    df['연월일'] = df['연월일'].astype(str)
    df['년'] = df['연월일'].str[:4].astype(int)
    df['월'] = df['연월일'].str[5:7].astype(int)
    df['일'] = df['연월일'].str[8:10].astype(int)

    df['예상'] = df['예상'].astype(str)
    df['예상_1'] = df['예상'].str[:2].astype(int)
    df['예상_2'] = df['예상'].str[3:].astype(int)

    df['도착'] = df['도착'].astype(str)
    df['도착_1'] = df['도착'].str[:2].astype(int)
    df['도착_2'] = df['도착'].str[3:].astype(int)

    df = df[['년', '월', '일', '항공사', '편명', '출발지', '예상_1', '예상_2', '도착_1', '도착_2', '현황']]
    # for col in list(set(df.columns) - set(categorical)):
    #     df = df.astypes({col:'float64'})
    
    categorical = ['항공사', '편명', '출발지', '현황']
    label_encoder = LabelEncoder()
    for col in categorical:
        df[col] = label_encoder.fit_transform(df[col])
        df = df.astype({col:'int'})

    df.info()

    return df


In [15]:
def calculate_f1_score(y_true, y_pred):
    # 정밀도, 재현율, F1 점수 계산
    # micro, macro, samples, weighted
    # weighted는 클래스의 빈도수에 따라 중요도를 부여하기 때문에, 샘플 수가 적은 클래스에 더 큰 가중치를 부여
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    return f1

## 0번째 데이터 (file name: train_airportal.csv, test_airportal.csv) 

## 년, 월, 일, 항공사, 편명, 출발지, 예상_1, 예상_2, 도착_1, 도착_2, 현황

In [39]:
data_train = pd.read_csv('./train_airportal.csv')
print(data_train['현황'].value_counts())
data_train = data_preprocessing(data_train)

data_test = pd.read_csv('./test_airportal.csv')
data_test = data_preprocessing(data_test)


x_train_tmp = data_train.iloc[:, :10]  # 0번부터 7번 열까지를 X로 설정
y_train_tmp = data_train.iloc[:, 10]   # 8번 열을 y로 설정

x_test_tmp = data_test.iloc[:, :10]  # 0번부터 7번 열까지를 X로 설정
y_test_tmp = data_test.iloc[:, 10]   # 8번 열을 y로 설정

print(x_test_tmp.info())
print(y_test_tmp.info())

출발    444548
지연     44655
취소       634
회항       199
Name: 현황, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 487397 entries, 0 to 490035
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487397 non-null  int64
 1   월       487397 non-null  int64
 2   일       487397 non-null  int64
 3   항공사     487397 non-null  int64
 4   편명      487397 non-null  int64
 5   출발지     487397 non-null  int64
 6   예상_1    487397 non-null  int64
 7   예상_2    487397 non-null  int64
 8   도착_1    487397 non-null  int64
 9   도착_2    487397 non-null  int64
 10  현황      487397 non-null  int64
dtypes: int64(11)
memory usage: 44.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 182019 entries, 0 to 184516
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       182019 non-null  int64
 1   월       182019 non-null  int64
 2   일       182019 non-null  int64
 3   항공사     18

In [40]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tmp, y_train_tmp)

y_pred = model.predict(x_test_tmp)
accuracy = accuracy_score(y_test_tmp, y_pred)

print("정확도:", accuracy)

정확도: 0.8218867261110104


In [41]:
print(calculate_f1_score(y_test_tmp, y_pred))

0.8588758767310122


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tmp, y_train_tmp)

y_pred = model.predict(x_test_tmp)
accuracy = accuracy_score(y_test_tmp, y_pred)

print("정확도:", accuracy)

정확도: 0.9342705981243716


In [43]:
print(calculate_f1_score(y_test_tmp, y_pred))

0.9265630879197277


## 1번째 데이터 (file name: 승객+편수16_18.csv, 승객+편수19.csv) 

## 예상_승객, 운행_편수 추가

In [12]:
first_train = pd.read_csv('승객+편수16_18.csv')
first_test = pd.read_csv('승객+편수19.csv')

first_train.info()
first_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487255 entries, 0 to 487254
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487255 non-null  int64
 1   월       487255 non-null  int64
 2   일       487255 non-null  int64
 3   항공사     487255 non-null  int64
 4   편명      487255 non-null  int64
 5   출발지     487255 non-null  int64
 6   예상_1    487255 non-null  int64
 7   예상_2    487255 non-null  int64
 8   현황      487255 non-null  int64
 9   예상_승객   487255 non-null  int64
 10  운항편수    487255 non-null  int64
dtypes: int64(11)
memory usage: 40.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181586 entries, 0 to 181585
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       181586 non-null  int64
 1   월       181586 non-null  int64
 2   일       181586 non-null  int64
 3   항공사     181586 non-null  int64
 4   편명      181586 non-null  int64
 5   출발지     18158

In [14]:
tmp1, tmp2 = first_train.iloc[:, :8], first_train.iloc[:, 9:]
x_train = pd.concat([tmp1, tmp2], axis=1)
y_train = first_train.iloc[:, 8]

tmp1, tmp2 = first_test.iloc[:, :8], first_test.iloc[:, 9:]
x_test = pd.concat([tmp1, tmp2], axis=1)
y_test = first_test.iloc[:, 8]

x_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487255 entries, 0 to 487254
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487255 non-null  int64
 1   월       487255 non-null  int64
 2   일       487255 non-null  int64
 3   항공사     487255 non-null  int64
 4   편명      487255 non-null  int64
 5   출발지     487255 non-null  int64
 6   예상_1    487255 non-null  int64
 7   예상_2    487255 non-null  int64
 8   예상_승객   487255 non-null  int64
 9   운항편수    487255 non-null  int64
dtypes: int64(10)
memory usage: 37.2 MB
<class 'pandas.core.series.Series'>
RangeIndex: 487255 entries, 0 to 487254
Series name: 현황
Non-Null Count   Dtype
--------------   -----
487255 non-null  int64
dtypes: int64(1)
memory usage: 3.7 MB


In [25]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.8947716233630346


In [26]:
print(calculate_f1_score(y_test, y_pred))

0.8973532629422621


In [27]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9098829204894651


In [28]:
print(calculate_f1_score(y_test, y_pred))

0.9054245584171872


## 2번째 데이터 (file name:승객+편수+날씨16_18, 승객+편수+날씨19) 

## 실제 출발 시간 drop 적용

## 가시거리, 온도, 풍속, 평일, 금요일, 주말, 공휴일 추가

In [30]:
second_train = pd.read_csv('승객+편수+날씨16_18.csv')
second_test = pd.read_csv('승객+편수+날씨19.csv')

second_train.info()
second_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485851 entries, 0 to 485850
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       485851 non-null  int64  
 1   월       485851 non-null  int64  
 2   일       485851 non-null  int64  
 3   항공사     485851 non-null  int64  
 4   편명      485851 non-null  int64  
 5   출발지     485851 non-null  int64  
 6   예상_1    485851 non-null  int64  
 7   예상_2    485851 non-null  int64  
 8   현황      485851 non-null  int64  
 9   예상_승객   485851 non-null  int64  
 10  운항편수    485851 non-null  int64  
 11  Count   485851 non-null  int64  
 12  가시거리    485851 non-null  float64
 13  온도      485851 non-null  float64
 14  풍속      485851 non-null  float64
dtypes: float64(3), int64(12)
memory usage: 55.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181112 entries, 0 to 181111
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0  

In [31]:
tmp1, tmp2 = first_train.iloc[:, :8], first_train.iloc[:, 9:]
x_train = pd.concat([tmp1, tmp2], axis=1)
y_train = first_train.iloc[:, 8]

tmp1, tmp2 = first_test.iloc[:, :8], first_test.iloc[:, 9:]
x_test = pd.concat([tmp1, tmp2], axis=1)
y_test = first_test.iloc[:, 8]

x_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485851 entries, 0 to 485850
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       485851 non-null  int64  
 1   월       485851 non-null  int64  
 2   일       485851 non-null  int64  
 3   항공사     485851 non-null  int64  
 4   편명      485851 non-null  int64  
 5   출발지     485851 non-null  int64  
 6   예상_1    485851 non-null  int64  
 7   예상_2    485851 non-null  int64  
 8   예상_승객   485851 non-null  int64  
 9   운항편수    485851 non-null  int64  
 10  Count   485851 non-null  int64  
 11  가시거리    485851 non-null  float64
 12  온도      485851 non-null  float64
 13  풍속      485851 non-null  float64
dtypes: float64(3), int64(11)
memory usage: 51.9 MB
<class 'pandas.core.series.Series'>
RangeIndex: 485851 entries, 0 to 485850
Series name: 현황
Non-Null Count   Dtype
--------------   -----
485851 non-null  int64
dtypes: int64(1)
memory usage: 3.7 MB


In [35]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9292813286805954


In [36]:
print(calculate_f1_score(y_test, y_pred))

0.9090970572550255


In [37]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.8769159415168515


In [38]:
print(calculate_f1_score(y_test, y_pred))

0.8860154688359566


In [9]:
lgb_classifier = lgb.LGBMClassifier()
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [2, 5, 15],
    'num_leaves': [31, 63, 127],
    'learning_rate': [0.01, 0.05],
    'min_child_samples': [100, 200, 250],
}

# 그리드 서치를 학습 데이터에 적용하고 최적의 하이퍼파라미터를 찾기
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print('Best parameters found by grid search:', grid_search.best_params_)

# 최적의 모델로 예측 및 정확도 측정
y_pred = grid_search.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy with best parameters:', accuracy)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found by grid search: {'learning_rate': 0.05, 'max_depth': 15, 'min_child_samples': 100, 'n_estimators': 200, 'num_leaves': 127}
Test accuracy with best parameters: 0.9361310301546312
